<div class="alert alert-block alert-info">

# Import libraries and data

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
price_df = pd.read_csv('Data/price.csv')
pricepersqft_df = pd.read_csv('Data/pricepersqft.csv')
price_df.head()

<div class="alert alert-block alert-info">

# Look for a Particular City

### Select a city

In [ ]:
# Check to see if the city is in the dataframe

city = "Phoenix"
check = price_df['City'].unique()
city in check

### Does the city only appear once?

In [ ]:
# Inspect the df to make sure there's one row

price_df.loc[price_df['City'] == city]

### Prep the city's data

In [ ]:
# Get the index of the city
city_index = price_df.index[price_df['City'] == city][0]

# Flip the dimensions of the dataframe
city_df = price_df.loc[city_index].transpose()

# Create the time column
monthly_intervals = pd.date_range('2010-11', periods=75, freq='M')

# Drop unnecessary columns 
city_df = city_df.drop(['City Code', 'City','Metro','County','State','Population Rank'], axis=0)
city_df = city_df.to_frame()

# Add the time column and set it as the index
city_df['Time'] = monthly_intervals.values
city_df = city_df.set_index('Time')

# Rename the Price column
city_df['Price'] = city_df[city_index]
city_df = city_df.drop([city_index], axis = 1)

# Create a list of prices.  This will be used to create the log column
city_prices = list(city_df['Price'])

<div class="alert alert-block alert-info">

# Stationarity Check Function

We will be using this function to check the stationarity of our data by performing a Dickie Fuller Test

In [ ]:
def stationarity_check(TS):

    # Calculate rolling statistics
    rolmean = TS.rolling(window = 12, center = False).mean()
    rolstd = TS.rolling(window = 12, center = False).std()
    
    # Perform the Dickey Fuller Test
    dftest = adfuller(TS) 
    
    #Plot rolling statistics:
    fig = plt.figure(figsize=(12,6))
    orig = plt.plot(TS, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    # Print Dickey-Fuller test results
    print ('Results of Dickey-Fuller Test:')

    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
    
    return None

<div class="alert alert-block alert-info">

# Transformations

### Look for which transformation has the lowest test statistic

In [ ]:
print('Log_Scale')
city_df['Log_Scale'] = np.log(city_prices)
stationarity_check(city_df.Log_Scale.dropna(inplace=False))

print('')
print('First_Difference')
city_df['First_Difference'] = city_df['Log_Scale'] - city_df['Log_Scale'].shift(1)
stationarity_check(city_df['First_Difference'].dropna(inplace=False) )

print('')
print('Rolling_Avg')
city_df['Rolling_Avg'] = city_df['Log_Scale'].rolling(window = 12).mean()
stationarity_check(city_df.Rolling_Avg.dropna(inplace=False))

print('')
print('Log_Sub_Rolling')
city_df['Log_Sub_Rolling'] = city_df['Log_Scale'][13:] - city_df['Rolling_Avg'][13:]
stationarity_check(city_df.Log_Sub_Rolling.dropna(inplace=False))

print('')
print('Exp_Weight_Avg')
city_df['Exp_Weight_Avg'] = city_df['Log_Scale'].ewm(halflife = 12, min_periods = 0, adjust = True).mean()
stationarity_check(city_df.Exp_Weight_Avg.dropna(inplace=False))

print('')
print('Log_Sub_Exp')
city_df['Log_Sub_Exp'] = city_df['Log_Scale'] - city_df['Exp_Weight_Avg']
stationarity_check(city_df.Log_Sub_Exp.dropna(inplace=False))

print('')
print('Log_Sub_Exp_Shift')
city_df['Log_Sub_Exp_Shift'] = city_df['Log_Sub_Exp'] - city_df['Log_Sub_Exp'].shift(1)
stationarity_check(city_df.Log_Sub_Exp_Shift.dropna(inplace=False))

print('')
print('Log_Sub_Roll_Shift')
city_df['Log_Sub_Roll_Shift'] = city_df['Log_Sub_Rolling'] - city_df['Log_Sub_Rolling'].shift(1)
stationarity_check(city_df.Log_Sub_Roll_Shift.dropna(inplace=False))

<div class="alert alert-block alert-info">

# Plot ACF and PACF graphs to get Q and P values

In [ ]:
from statsmodels.tsa.stattools import acf, pacf

# ACF and PACF plots:
# We use the column that produced the best stationarity

### This is a section that needs to be fill out: ### 

best_dickie_fuller = 'Log_Sub_Exp_Shift'

city_df[best_dickie_fuller].dropna(inplace = True)
lag_acf = acf(city_df[best_dickie_fuller], nlags=10)
lag_pacf = pacf(city_df[best_dickie_fuller], nlags=10, method='ols')


#Plot ACF: 
plt.figure(figsize = (12,6))
plt.subplot(211) 

plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(74),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(74),linestyle='--',color='gray')
plt.title('Autocorrelation Function')
plt.tight_layout()

#Plot PACF:
plt.figure(figsize = (12,6))
plt.subplot(212)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(74),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(74),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

<div class="alert alert-block alert-info">

# Build the ARIMA Model

When modeling, we model against the Log Scale and tune the parameters based on the ACF and PACF graphs

In [ ]:
# We model against the Log Scale
X = city_df['Log_Scale']

# How big do we want our test set?
size = int(len(X) * 0.66)

# Train, Test, Split
train, test = X[0:size], X[size:len(X)]

# Want just a list of the training values, no time attached
history = [x for x in train]

# print each prediction
predictions = []
for t in range(len(test)):
    model = ARIMA(history, order=(4,2,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted = %f, expected = %f, absolute difference = %f' % (yhat, obs, (abs(yhat - obs))))
error = mean_squared_error(test, predictions)
print('Test MSE: %.9f' % error)

In [ ]:
# Create predictions
predicts = []

# Convert the predicted log values to rental prices
for i in predictions:
    predicts.extend(np.exp(i))

# Convert the expected log value to rental prices
expects = test.tolist()
expects_median = []
for i in expects:
    expects_median.append(np.exp(i))


In [ ]:
# Graph the expected and the predicted values

plt.figure(figsize = (14,7))
x = np.arange(0, 26, 1)
# create an index for each tick position
xi = [i for i in range(0, len(x))]
y = predicts
z = expects_median
plt.plot(xi, y, linestyle='-', color='r', label='Predicted', linewidth=4.0)
plt.plot(xi, z, linestyle='-', color='b', label='Actual', linewidth=4.0) 
plt.xlabel('Last 26 Months', fontsize=24)
plt.ylabel('Median Rental Price', fontsize=24) 
plt.xticks(xi, x,)
plt.tick_params('x', colors='black', size = 10, labelsize = 14)
plt.tick_params('y', colors='black', size = 10, labelsize = 14)
plt.title(city, fontsize=30)
plt.legend(prop={'size': 20}) 
plt.show()

<div class="alert alert-block alert-info">

# MAPE and Mean Squared Error

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    print('Mape = ' + str(np.mean(np.abs((y_true - y_pred) / y_true)) * 100))
mean_absolute_percentage_error(expects_median, predicts)
print('MSE = ' + str(mean_squared_error(expects_median, predicts)))

<div class="alert alert-block alert-info">

# Predicting the next year

In [ ]:
# Get the foecast for the nex 12 mntohs
forecast = model_fit.forecast(steps = 12)

# Next 12 month predictions
next_12_months = forecast[0].tolist()
next_year = []
for i in next_12_months:
    next_year.append(np.exp(i))
next_year

In [ ]:
# Upper Confidence Boundry
forecast = model_fit.forecast(steps = 12)
next_12_months = (forecast[0] + forecast[1]).tolist()
upper_ci = []
for i in next_12_months:
    upper_ci.append(np.exp(i))
upper_ci

In [ ]:
# Lower Confidence Boundry
forecast = model_fit.forecast(steps = 12)
next_12_months = (forecast[0] - forecast[1]).tolist()
lower_ci = []
for i in next_12_months:
    lower_ci.append(np.exp(i))
lower_ci

In [ ]:
plt.figure(figsize = (14,7))
x = np.arange(0, 38, 1)
# create an index for each tick position
predict_x = [i for i in range(26, len(x))]
xi = [i for i in range(0, len(x)-12)]
y = predicts
z = expects_median

plt.plot(xi, y, linestyle='-', color='r', label='Predicted', linewidth=4.0)
plt.plot(xi, z, linestyle='-', color='b', label='Actual', linewidth=4.0)
plt.plot(predict_x, next_year, linestyle='-', color='green', label='Next Year', linewidth=4.0)

# confidence intervals
plt.plot(predict_x, lower_ci, linestyle='-', color='gray', label='CI', linewidth=4.0)
plt.plot(predict_x, upper_ci, linestyle='-', color='gray', linewidth=4.0)

plt.xlabel('Months', fontsize=24)
plt.ylabel('Median Rental Price', fontsize=24) 
plt.xticks(x, x,)
plt.tick_params('x', colors='black', size = 10, labelsize = 14)
plt.tick_params('y', colors='black', size = 10, labelsize = 14)
plt.title(city, fontsize=30)
plt.legend(prop={'size': 20}) 
plt.show()